## Install & Import bibliothèque

In [1]:
!pip install python-lichess

In [2]:
!pip install python-chess

In [3]:
!pip install pySerial

In [4]:
!pip install berserk

In [5]:
import lichess.api

In [6]:
import berserk

In [7]:
import requests

In [8]:
import time

In [9]:
import re
# La fonction re.sub() de la bibliothèque re permet de remplacer toutes les occurrences d'un motif de recherche par une chaîne de remplacement. Dans ce cas, le motif de recherche est "\{.*?\}", qui correspond à toute chaîne de caractères qui commence par { et se termine par }. La chaîne de remplacement est simplement une chaîne vide, "", ce qui permet de supprimer ces parties de la chaîne d'origine.

In [10]:
import webbrowser

## TOKEN Lichess 

In [11]:
# BOTKAIROS : lip_ndcKmSqJoPKlp6H3yDV1
#lip_09gpe64RAdOHUTBN0CmK

token = "lip_09gpe64RAdOHUTBN0CmK"
session = berserk.TokenSession(token)
client = berserk.Client(session=session)

### Board 

In [12]:
DisplayBoard = [    ["R1", "N1", "B1", "Q", "K", "B2", "N2", "R2"],
    ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"],
    ["r1", "n1", "b1", "q", "k", "b2", "n2", "r2"]
]

def display_board(board):
    print("  A  B  C  D  E  F  G  H")
    for i, row in enumerate(board):
        print(8-i, end=" ")
        for square in row:
            if (len(square) == 2) :
                print(square, end=" ")
            else :
                print(square, end="  ")
        print()
    print("  A  B  C  D  E  F  G  H")

display_board(DisplayBoard)



  A  B  C  D  E  F  G  H
8 R1 N1 B1 Q  K  B2 N2 R2 
7 P1 P2 P3 P4 P5 P6 P7 P8 
6                         
5                         
4                         
3                         
2 p1 p2 p3 p4 p5 p6 p7 p8 
1 r1 n1 b1 q  k  b2 n2 r2 
  A  B  C  D  E  F  G  H


In [13]:
ligne = ["1","2","3","4","5","6","7","8"]
colonne = ['a','b','c','d','e','f','g','h']

### Create IA  

In [14]:
def IA_Level():
    while True:
        input_ia = int(input("Saisir le niveau de difficulté de l'IA (1 à 8) : "))
        if input_ia < 1 or input_ia > 8:
            print("Erreur : la valeur doit être comprise entre 1 et 8.")
        else:
            print(f"\nIA de difficulté {input_ia}/8.\n")
            return input_ia

In [15]:
def ColorPlayer():
    while True:
        color_player = input("Saisir la couleur du joueur (white or black): ")
        if color_player.lower() not in ["white", "black"]:
            print("Erreur : la couleur doit être white ou black.")
        else:
            print(f"\nVous avez choisi la couleur : {color_player}.\n")
            return color_player.lower()

In [16]:
def CreateIA() :
    ia_level = IA_Level()
    color_player = ColorPlayer()
    try :
        print("Création de la game contre l'IA")
        id_game = client.challenges.create_ai(level=ia_level, clock_limit=None, clock_increment=None, days=None, color=color_player, variant=None, position=None)['id']
        if (color_player == "white"):
            color_enemy = "black"
        else :
            color_enemy = "white"
        url = f'https://lichess.org/{id_game}'
        webbrowser.open(url)
        return id_game, color_player, color_enemy
    except :
        print("Error")
        CreateIA()

In [17]:
# Fonction pour obtenir les mouvements en direct de la partie
def LiveMove(game_id):
    response = requests.get(f'https://lichess.org/{game_id}')
    PGN = response.text
    PGN_String = str(PGN)
    start = PGN_String.find("1. ")
    substring = PGN_String[start:]
    PGN_String = substring
    moves = PGN_String.split("</div>")[0]
    return moves

In [18]:
board = [    ["R1", "N1", "B1", "Q", "K", "B2", "N2", "R2"],
    ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"],
    ["r1", "n1", "b1", "q", "k", "b2", "n2", "r2"]
]

previousmove = "a2"


board[ligne.index(previousmove[1])][colonne.index(previousmove[0])][0] == "P"

True

In [19]:
def InputPos():
    while True:
        move = input("Saisir la position du mouvement (ex: e2e4 ou g8f6) : ")
        if len(move) not in [4, 5]:
            print("Erreur : le mouvement doit comporter 4 ou 5 caractères.")
        elif not move[0].isalpha() or not move[2].isalpha():
            print("Erreur : les caractères 1 et 3 doivent être des lettres.")
        elif not move[1].isdigit() or not move[3].isdigit():
            print("Erreur : les caractères 2 et 4 doivent être des chiffres.")
        elif len(move) == 5 and not move[4].isalpha():
            print("Erreur : le dernier caractère doit être une lettre.")
        else:
            return move.lower()

In [20]:
def InputMove(id_game, board, color_player):
    input_move = InputPos()
    finalmove = ''
    print("\n")
    try:
        if (color_player == "white") :
            # Petit roque
            if input_move == "e1h1" and board[0][4] == "K" and board[0][7] == "R2" and board[0][5] == ' ' and board[0][6] == ' ':
                board[0][4] = " "
                board[0][7] = " "
                board[0][6] = "K"
                board[0][5] = "R2"
                finalmove = "h1f1 e1g1"
                client.bots.make_move(id_game, input_move)
                return finalmove, board
            # Grand roque
            elif input_move == "e1a1" and board[0][4] == "K" and board[0][0] == "R1" and board[0][1] == ' ' and board[0][2] == ' ' and board[0][3] == ' ':    
                board[0][0] = " "
                board[0][4] = " "
                board[0][3] = "R2"
                board[0][2] = "K"
                finalmove = "a1d1 e1c1"
                client.bots.make_move(id_game, input_move)
                return finalmove, board
            elif (input_move[1] == "7" and input_move[3] == "8") :
                move = input_move[2:]
                previousmove = input_move[:2]
                if (board[ligne.index(previousmove[1])][colonne.index(previousmove[0])][0] == "P") :
                    input_piece = input("=> Saisir la nouvelle valeur de la pièce (N : Knight, B : Bishop, Q : Queen, R : Rook) : ")
                    input_move = input_move + input_piece.lower()
                    client.bots.make_move(id_game, input_move)  
                    board[ligne.index(move[1])][colonne.index(move[0])] = input_piece
                    board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                    return finalmove, board
                else :
                    client.bots.make_move(id_game, input_move)
                    move = input_move[2:]
                    previousmove = input_move[:2]
                    # Si un pion est mangé ou non 
                    if board[ligne.index(move[1])][colonne.index(move[0])] != ' ':
                        finalmove = move + "stock " + input_move
                    else:
                        finalmove = input_move
                    # Définition du plateau
                    lastboard = board
                    board[ligne.index(move[1])][colonne.index(move[0])] = lastboard[ligne.index(previousmove[1])][colonne.index(previousmove[0])]
                    board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                    return finalmove, board
            else:
                client.bots.make_move(id_game, input_move)
                move = input_move[2:]
                previousmove = input_move[:2]
                # Si un pion est mangé ou non 
                if board[ligne.index(move[1])][colonne.index(move[0])] != ' ':
                    finalmove = move + "stock " + input_move
                else:
                    finalmove = input_move
                # Définition du plateau
                lastboard = board
                board[ligne.index(move[1])][colonne.index(move[0])] = lastboard[ligne.index(previousmove[1])][colonne.index(previousmove[0])]
                board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                return finalmove, board
        elif (color_player == "black") :
            # Petit roque
            if input_move == "e8h8" and board[7][4] == "K" and board[7][7] == "R2" and board[7][5] == ' ' and board[7][6] == ' ':
                board[7][4] = " "
                board[7][7] = " "
                board[7][6] = "K"
                board[7][5] = "R2"
                finalmove = "h8f8 e8g8"
                client.bots.make_move(id_game, input_move)
                return finalmove, board
            # Grand roque
            elif input_move == "e1a1" and board[7][4] == "K" and board[7][0] == "R1" and board[7][1] == ' ' and board[7][2] == ' ' and board[7][3] == ' ':    
                board[7][0] = " "
                board[7][4] = " "
                board[7][3] = "R2"
                board[7][2] = "K"
                finalmove = "a8d8 e8c8"
                client.bots.make_move(id_game, input_move)
                return finalmove, board
            elif (input_move[1] == "2" and input_move[3] == "1") :
                move = input_move[2:]
                previousmove = input_move[:2]
                if (board[ligne.index(previousmove[1])][colonne.index(previousmove[0])][0] == "p") :
                    input_piece = input("=> Saisir la nouvelle valeur de la pièce (n : Knight, b : Bishop, q : Queen, r : Rook) : ")
                    input_move = move + "=" + input_piece
                    client.bots.make_move(id_game, input_move)  
                    board[ligne.index(move[1])][colonne.index(move[0])] = input_piece
                    board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                    return finalmove, board
                else :
                    client.bots.make_move(id_game, input_move)
                    move = input_move[2:]
                    previousmove = input_move[:2]
                    # Si un pion est mangé ou non 
                    if board[ligne.index(move[1])][colonne.index(move[0])] != ' ':
                        finalmove = move + "stock " + input_move
                    else:
                        finalmove = input_move
                    # Définition du plateau
                    lastboard = board
                    board[ligne.index(move[1])][colonne.index(move[0])] = lastboard[ligne.index(previousmove[1])][colonne.index(previousmove[0])]
                    board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                    return finalmove, board
            else:
                client.bots.make_move(id_game, input_move)
                move = input_move[2:]
                previousmove = input_move[:2]
                # Si un pion est mangé ou non 
                if board[ligne.index(move[1])][colonne.index(move[0])] != ' ':
                    finalmove = move + "stock " + input_move
                else:
                    finalmove = input_move
                # Définition du plateau
                lastboard = board
                board[ligne.index(move[1])][colonne.index(move[0])] = lastboard[ligne.index(previousmove[1])][colonne.index(previousmove[0])]
                board[ligne.index(previousmove[1])][colonne.index(previousmove[0])] = ' '
                return finalmove, board
    except:
        print("\nErreur position")
        input_move = InputMove(id_game, board)
    
    return finalmove, board


In [21]:
def possible_moves(piece, current_position, color_player):
    row, col = current_position
    if (row >= 0 and row <= 7 and col >= 0 and col <= 7) :
        possible_positions = []
        if piece == 'pawn':
            if (color_player == "white") :
                if (row == 3) :
                    possible_positions.append((row - 1, col))
                    possible_positions.append((row - 2, col))
                else:
                    possible_positions.append((row - 1, col))
                possible_positions.append((row - 1, col+1))
                possible_positions.append((row - 1, col-1))
            else :
                if (row == 4) :
                    possible_positions.append((row + 1, col))
                    possible_positions.append((row + 2, col))
                else:
                    possible_positions.append((row + 1, col))
                possible_positions.append((row + 1, col+1))
                possible_positions.append((row + 1, col-1))
                
        elif piece == 'knight':
            possible_positions.append((row + 2, col + 1))
            possible_positions.append((row + 2, col - 1))
            possible_positions.append((row - 2, col + 1))
            possible_positions.append((row - 2, col - 1))
            possible_positions.append((row + 1, col + 2))
            possible_positions.append((row + 1, col - 2))
            possible_positions.append((row - 1, col + 2))
            possible_positions.append((row - 1, col - 2))
        elif piece == 'bishop':
            for i in range(1, 8):
                possible_positions.append((row + i, col + i))
                possible_positions.append((row - i, col - i))
                possible_positions.append((row + i, col - i))
                possible_positions.append((row - i, col + i))
        elif piece == 'rook':
            for i in range(1, 8):
                possible_positions.append((row, col + i))
                possible_positions.append((row, col - i))
                possible_positions.append((row + i, col))
                possible_positions.append((row - i, col))
        elif piece == 'queen':
            for i in range(1, 8):
                possible_positions.append((row, col + i))
                possible_positions.append((row, col - i))
                possible_positions.append((row + i, col))
                possible_positions.append((row - i, col))
                possible_positions.append((row + i, col + i))
                possible_positions.append((row - i, col - i))
                possible_positions.append((row + i, col - i))
                possible_positions.append((row - i, col + i))
        elif piece == 'king':
            possible_positions.append((row, col + 1))
            possible_positions.append((row, col - 1))
            possible_positions.append((row + 1, col))
            possible_positions.append((row - 1, col))
            possible_positions.append((row + 1, col + 1))
            possible_positions.append((row - 1, col - 1))
            possible_positions.append((row + 1, col - 1))
            possible_positions.append((row - 1, col + 1))
    
        
        return [pos for pos in possible_positions if all(0 <= coord <= 7 for coord in pos)]

    else : 
        return 'Error'

In [22]:
def ConversionMouv(positions):    
    positionfinal = []
    for i in range(len(positions)):
        positionfinal.append(str(colonne[positions[i][1]]) + str(ligne[positions[i][0]]))
    return positionfinal

In [23]:
# Fonction pour obtenir les mouvements en direct de la partie
def LiveMove(game_id):
    response = requests.get(f'https://lichess.org/{game_id}')
    PGN = response.text
    PGN_String = str(PGN)
    start = PGN_String.find("1. ")
    substring = PGN_String[start:]
    PGN_String = substring
    moves = PGN_String.split("</div>")[0]
    moves = re.sub(r"\{.*?\}", "", moves)
    return moves

In [24]:
# Exemple d'utilisation de la fonction
#id_game = "pdYClRba"
id_game = "ztXyemyUJfDd"
id_game = "t7vSZZvPOjrt"
id_game = "ZgnhQZS9"
ListMove = LiveMove(id_game)

print(ListMove)

1. a4 e6 2. Nc3 Qf6 3. h4 Nc6 4. Rh3 d5 5. Rf3 Qe7 6. b4 Ne5 7. Nxd5 Qd6 8. Bb2 Ne7 9. Bxe5 exd5 *


In [25]:
def MouvEnnemy(tour, color, ListMove):
    if (color == "white") :
        Sep_LastMove = str(tour) +". "
        StartLastMove = ListMove.find(Sep_LastMove)
        SubstringLastMove = ListMove[StartLastMove + len(Sep_LastMove):]
        moves = SubstringLastMove.split(" *")[0]
        moves = moves.split(" ")
        moves = moves[1]
        moves = moves.replace(" ", "")
        return moves
    else :
        Sep_LastMove = str(tour) +". "
        StartLastMove = ListMove.find(Sep_LastMove)
        SubstringLastMove = ListMove[StartLastMove:]
        moves = SubstringLastMove.split(" *")[0]
        moves = moves.split(" ")
        moves = moves[1] 
        moves = moves.replace(" ", "")
        return moves

In [26]:
def extraire_mouvement(chaine):
    mouvement = re.search(r"[a-h][1-8]", chaine)
    if mouvement:
        return mouvement.group()
    else:
        return None

In [27]:
def PositionFinal(move, board, color_player, positions, piece, letter) :    
    initial_position = ""
    for i in range(len(positions)) :
        if (color_player == "white" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
        elif (color_player == "black" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter.lower()) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
            
    initial_position = str(positions[i][0]) + str(positions[i][1])
    finalmove = initial_position + extraire_mouvement(move)
    return finalmove, board

In [28]:
def PositionFinal4(move, board, color_player, positions, piece, letter) :    
    initial_position = ""
    depart = move[1]
    move = move[0:1] + move[2:]
    
    lastposition = positions
    positions = []

    if isinstance(depart, str):
        for element in lastposition:
            if element.startswith(depart):
                positions.append(element)
    else : 
        for element in lastposition:
            if element[1] == depart:
                positions.append(element)
        
    for i in range(len(positions)) :
        if (color_player == "white" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
        elif (color_player == "black" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter.lower()) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
            
    initial_position = str(positions[i][0]) + str(positions[i][1])
    finalmove = initial_position + extraire_mouvement(move)
    return finalmove, board

In [29]:
def PositionFinalX(move, board, color_player, positions, piece, letter) :   
    move = move.replace("x", "")
    initial_position = ""
    for i in range(len(positions)) :
        if (color_player == "white" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
        elif (color_player == "black" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter.lower()) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
            
    initial_position = str(positions[i][0]) + str(positions[i][1])
    finalmove = extraire_mouvement(move) + "stock " + initial_position + extraire_mouvement(move)
    return finalmove, board

In [30]:
def PositionFinalX4(move, board, color_player, positions, piece, letter) :    
    move = move.replace("x","")
    initial_position = ""
    depart = move[2]
    move = move[0:1] + move[2:]
    
    lastposition = positions
    positions = []

    if isinstance(depart, str):
        for element in lastposition:
            if element.startswith(depart):
                positions.append(element)
    else : 
        for element in lastposition:
            if element[1] == depart:
                positions.append(element)
                
    for i in range(len(positions)) :
        if (color_player == "white" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
        elif (color_player == "black" and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] != ' ' and board[ligne.index(positions[i][1])][colonne.index(positions[i][0])][0] == letter.lower()) :
            piece = board[ligne.index(positions[i][1])][colonne.index(positions[i][0])]
            if (len(move) == 2):
                board[ligne.index(move[1])][colonne.index(move[0])] = piece
            elif (len(move) == 3) :
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            else : 
                board[ligne.index(move[2])][colonne.index(move[1])] = piece
            board[ligne.index(positions[i][1])][colonne.index(positions[i][0])] = ' '
            break
            
    initial_position = str(positions[i][0]) + str(positions[i][1])
    finalmove = extraire_mouvement(move) + "stock " + initial_position + extraire_mouvement(move)
    return finalmove, board

In [31]:
def ROQUE(board, move, color_player):
    """
    Fonction pour réaliser un roque (grand ou petit) en fonction de la notation FEN `O-O` (petit roque) ou `O-O-O` (grand roque)

    Args:
    - board (list): Liste de listes représentant l'état actuel du plateau de jeu.
    - move (str): Notation FEN du roque à réaliser, soit 'O-O' pour le petit roque, soit 'O-O-O' pour le grand roque.

    Returns:
    - board (list): Liste de listes représentant le nouvel état du plateau de jeu après le roque.
    """
    if move == 'O-O':
        if (color_player == "white") :
            # Vérification des conditions pour le petit roque
            if board[0][4] == 'K' and board[0][7] == 'R2':
#                for i in range(5, 7):
#                    if board[0][i] != ' ':
#                        # Si une case entre le roi et la tour n'est pas vide, on ne peut pas roquer
#                        return board
                # Les cases entre le roi et la tour sont vides, on peut réaliser le roque
                board[0][6] = 'K'
                board[0][5] = 'R2'
                board[0][4] = ' '
                board[0][7] = ' '
                return "a8a6 a5a7", board
        if (color_player == "black") :
                # Vérification des conditions pour le petit roque
                if board[7][4] == 'k' and board[7][7] == 'r2':
#                    for i in range(5, 7):
#                        if board[7][i] != ' ':
#                            # Si une case entre le roi et la tour n'est pas vide, on ne peut pas roquer
#                            return board
                    # Les cases entre le roi et la tour sont vides, on peut réaliser le roque
                    board[7][6] = 'k'
                    board[7][5] = 'r2'
                    board[7][4] = ' '
                    board[7][7] = ' '
                    return "h8h6 h5h7", board
    elif move == 'O-O-O':
        if (color_player == "white") :
            # Vérification des conditions pour le grand roque
            if board[0][4] == 'K' and board[0][0] == 'R1':
#                for i in range(1, 4):
#                    if board[0][i] != ' ':
#                        # Si une case entre le roi et la tour n'est pas vide, on ne peut pas roquer
#                        return board
                # Les cases entre le roi et la tour sont vides, on peut réaliser le roque
                board[0][2] = 'K'
                board[0][3] = 'R1'
                board[0][4] = ' '
                board[0][0] = ' '
                return "a1a4 a5a3", board
        if (color_player == "black") :
            # Vérification des conditions pour le grand roque
            if board[7][4] == 'k' and board[7][0] == 'r1':
#                for i in range(1, 4):
#                    if board[7][i] != ' ':
#                        # Si une case entre le roi et la tour n'est pas vide, on ne peut pas roquer
#                        return board
                # Les cases entre le roi et la tour sont vides, on peut réaliser le roque
                board[7][2] = 'k'
                board[7][3] = 'r1'
                board[7][4] = ' '
                board[7][0] = ' '
                return "h1h4 h5h3", board
    # Si aucun roque n'est possible, on renvoie le plateau de jeu inchangé
    return "", board

In [32]:
def PossibleMove(move, board, color_player):
    move = move.replace("+","")
    if (len(move) == 2):
        # Nous avons donc un pion
        piece = 'pawn'
        letter = 'P'
        positions = ConversionMouv(possible_moves(piece, (ligne.index(move[1]),colonne.index(move[0])), color_player))
        finalmove, board = PositionFinal(move, board, color_player, positions, piece, letter)
        return finalmove, board
    
    elif (move == "O-O" or move == "O-O-O") :
        finalmove, board = ROQUE(board, move, color_player)
        return finalmove, board
        
    else :
        letter = move[0]
        if (letter == "N") :
            piece = 'knight'
        elif (letter == "B") :
            piece = 'bishop'
        elif (letter == "R") :
            piece = 'rook'
        elif (letter == "Q") :
            piece = 'queen'
        elif (letter == "K") :
            piece = 'king'
        else :
            piece = 'pawn'
            
        if 'x' not in move : 
            if (len(move) == 3): 
                positions = ConversionMouv(possible_moves(piece, (ligne.index(move[2]),colonne.index(move[1])), color_player))
                finalmove, board = PositionFinal(move, board, color_player, positions, piece, letter)
                return finalmove, board
            else :   
                positions = ConversionMouv(possible_moves(piece, (ligne.index(move[3]),colonne.index(move[2])), color_player))
                finalmove, board = PositionFinal4(move, board, color_player, positions, piece, letter)
                return finalmove, board
        else :
            move = move.replace("x","")
            if (len(move) == 3): 
                positions = ConversionMouv(possible_moves(piece, (ligne.index(move[2]),colonne.index(move[1])), color_player))
                finalmove, board = PositionFinalX(move, board, color_player, positions, piece, letter)
                return finalmove, board
            else :   
                positions = ConversionMouv(possible_moves(piece, (ligne.index(move[3]),colonne.index(move[2])), color_player))
                finalmove, board = PositionFinalX4(move, board, color_player, positions, piece, letter)
                return finalmove, board
        return "Error"
    return "Error"


# _______________________________

# _______________________________

# _______________________________

In [33]:
import serial

In [34]:
DisplayBoard = [    ["R1", "N1", "B1", "Q", "K", "B2", "N2", "R2"],
    ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"],
    ["r1", "n1", "b1", "q", "k", "b2", "n2", "r2"]
]
tour = 1

def Game(DisplayBoard, tour) :
    print("---------------------------------------")
    print("\nCréation de la partie contre une IA\n")
    print("---------------------------------------")
    id_game, color_player, color_enemy = CreateIA()
    print("Identifiant de la game : ", id_game)
    EndGame = False
    while (EndGame == False) :
        if (color_player == "white") :
            finalmove_player, DisplayBoard = InputMove(id_game, DisplayBoard,color_player)
            print(f"Commande Arduino : {finalmove_player}")
            print("\n\n", display_board(DisplayBoard), "\n\n")
            time.sleep(7)
            ListMove = LiveMove(id_game)    
            if any(result in ListMove for result in ["0-1", "1-0", "0,5-0,5", "1/2-1/2"]):
                EndGame = True
                break
            move = MouvEnnemy(tour, color_player, ListMove)
            finalmove, DisplayBoard = PossibleMove(move, DisplayBoard, color_enemy)
            print(f"\nEnnemy movement : {finalmove}\n")
            print("\n\n", display_board(DisplayBoard), "\n\n")
            time.sleep(7)
        else :
            time.sleep(7)
            ListMove = LiveMove(id_game)
            move = MouvEnnemy(tour, color_player, ListMove)
            finalmove, DisplayBoard = PossibleMove(move, DisplayBoard, color_enemy)
            print(f"\nEnnemy movement : {finalmove}\n")
            print("\n\n", display_board(DisplayBoard), "\n\n")
            time.sleep(7)
            ListMove = LiveMove(id_game)    
            if any(result in ListMove for result in ["0-1", "1-0", "0,5-0,5", "1/2-1/2"]):
                EndGame = True
                break
            finalmove_player, DisplayBoard = InputMove(id_game, DisplayBoard, color_player)
            print("\n\n", display_board(DisplayBoard), "\n\n")
            time.sleep(7)
            
        ListMove = LiveMove(id_game)    
        if any(result in ListMove for result in ["0-1", "1-0", "0,5-0,5", "1/2-1/2"]):
            EndGame = True
        
        tour = tour + 1
    
    ListMove = LiveMove(id_game) 
    print("\n\n---------------")
    print("Fin de la game.")
    print("---------------\n\n")
    if "0-1" in ListMove:
        print("\nVictoire des noirs")
    elif "1-0" in ListMove: 
        print("\nVictoire des blancs")
    else :
        print("\nMatch Nul")
    
    print(f"\n\nNombre de tours joués : {tour}")
        

    
        
        

In [35]:
DisplayBoard = [    ["R1", "N1", "B1", "Q", "K", "B2", "N2", "R2"],
    ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8"],
    ["r1", "n1", "b1", "q", "k", "b2", "n2", "r2"]
]
tour = 1
Game(DisplayBoard, tour)

---------------------------------------

Création de la partie contre une IA

---------------------------------------
Saisir le niveau de difficulté de l'IA (1 à 8) : 8

IA de difficulté 8/8.

Saisir la couleur du joueur (white or black): white

Vous avez choisi la couleur : white.

Création de la game contre l'IA
Identifiant de la game :  BaoJrg5K


KeyboardInterrupt: Interrupted by user

In [ ]:
LiveMove("mZfCscOr")

In [ ]:
MouvEnnemy(8, "white", LiveMove("mZfCscOr"))

In [ ]:
ListMove = LiveMove("vYceLdqw")
print(ListMove)
if any(result in ListMove for result in ["0-1", "1-0", "0,5-0,5", "1/2-1/2"]):
    print("oui")
else :
    print("non")

In [ ]:
LiveMove("9kFvKBtt")

In [ ]:
LiveMove("xN9SrAaV")

# _______________________________

# _______________________________

# _______________________________

# _______________________________

# _______________________________

# _______________________________

# ____________________________________________________________________________________________________________________________________________________________________________________________________________


# ____________________________________________________________________________________________________________________________________________________________________________________________________________


# ____________________________________________________________________________________________________________________________________________________________________________________________________________
